# WiGLE API export - Test Analysis

Go to [WiGLE](https://wigle.net/) and in the toolbar, select **Tools** and then **Account**. Register or Login in order to obtain you API key. Then, in the toolbar select **Tools** and **API**. From here, you can use the Swagger UI to run queries (you must select **Authorize** and then enter your API information in the the User and Password sections) OR you can take your API key to interact directly with the API.

**Note:** A single query will only return 100 results and larger extracts require contacting WiGLE sys admins.

In [1]:
import pandas as pd
%matplotlib inline

In [2]:
wigle_lex = pd.read_json('lex.json')

In [3]:
wigle_lex.head(5)

,first,last,resultCount,results,search_after,success,totalResults
0,1,100,100,"{'housenumber': None, 'comment': None, 'city':...",56309,True,162722
1,1,100,100,"{'housenumber': None, 'comment': None, 'city':...",56309,True,162722
2,1,100,100,"{'housenumber': None, 'comment': None, 'city':...",56309,True,162722
3,1,100,100,"{'housenumber': None, 'comment': None, 'city':...",56309,True,162722
4,1,100,100,"{'housenumber': None, 'comment': None, 'city':...",56309,True,162722


Currently (as of 30 AUG 2017) 162722 total records for Lexington, KY (geofenced)

In [4]:
wigle_lex.results[0]

{'bcninterval': 0,
 'channel': 11,
 'city': None,
 'comment': None,
 'country': 'US',
 'dhcp': '?',
 'firsttime': '2011-05-27T16:50:52.000Z',
 'freenet': '?',
 'housenumber': None,
 'lasttime': '2013-05-28T03:31:31.000Z',
 'lastupdt': '2013-05-28T01:32:39.000Z',
 'name': None,
 'netid': '00:18:0A:21:18:31',
 'paynet': '?',
 'qos': 2,
 'region': 'KY',
 'road': None,
 'ssid': 'Motel 6',
 'transid': '20110527-00127',
 'trilat': 38.04119492,
 'trilong': -84.42301941,
 'type': 'infra',
 'userfound': False,
 'wep': 'N'}

Need to convert result column into data frame to access signal and location information

In [5]:
wifi_signals = wigle_lex['results'].apply(pd.Series)
wifi_signals.head(5)

,bcninterval,channel,city,comment,country,dhcp,firsttime,freenet,housenumber,lasttime,...,qos,region,road,ssid,transid,trilat,trilong,type,userfound,wep
0,0,11,None,None,US,?,2011-05-27T16:50:52.000Z,?,None,2013-05-28T03:31:31.000Z,...,2,KY,None,Motel 6,20110527-00127,38.041195,-84.423019,infra,False,N
1,0,11,None,None,None,?,2015-06-03T08:40:20.000Z,?,None,2015-06-03T08:30:25.000Z,...,0,None,None,Motel 6,20150603-00052,38.040455,-84.421707,infra,False,?
2,0,6,None,None,US,?,2010-03-05T07:13:26.000Z,?,None,2010-03-05T01:20:54.000Z,...,1,KY,None,linksys,20100304-00053,38.024086,-84.564728,infra,False,Y
3,0,1,None,None,US,?,2007-07-22T23:53:37.000Z,?,None,2007-07-22T19:11:32.000Z,...,1,KY,None,<no ssid>,20070721-00044,38.042881,-84.419693,????,False,N
4,0,6,None,None,US,?,2010-03-05T07:13:06.000Z,?,None,2010-03-05T01:20:53.000Z,...,1,KY,None,KALMnet,20100304-00053,38.021336,-84.569885,infra,False,Y


Mapping Lat/Long with SSID

In [6]:
import folium
from folium.plugins import MarkerCluster
from IPython.display import IFrame

In [7]:
locations = list(zip(wifi_signals.trilat, wifi_signals.trilong))
signal_map = folium.Map(location=[38.046385, -84.503192], tiles='Cartodb Positron', zoom_start=11)
signal_map.add_child(MarkerCluster(locations=locations, popups=wifi_signals["ssid"]))
signal_map